In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
def change_dtype_ser(ser):
    
    if ser.dtype == int:
        return ser.astype(np.int32)
    
    if ser.dtype == float:
        return ser.astype(np.float32)
    
    if ser.dtype == np.object:
        return ser.astype("category")
    
    return ser
    

def change_dtype_df(df):
    """
    change types of columns to reduce memory size
    :param df: dataframe
    :return df: dataframe
    """
    df["fecha_dato"] = pd.to_datetime(df["fecha_dato"])
    df["fecha_alta"] = pd.to_datetime(df["fecha_alta"])
    
    memory = df.memory_usage().sum() / 10**6
    print("Memory usage before changing types %0.2f MB" % memory)

    for col in df.columns:
        df[col] = change_dtype_ser(df[col])

    memory = df.memory_usage().sum() / 10 ** 6
    print("Memory usage after changing types %0.2f MB" % memory)
    return df


def load_csv(filename):
    df = pd.read_csv(filename)
    df = change_dtype_df(df)
    return df

In [3]:
INP_DIR = "data/data_"
OUT_DIR1 = "data/data1_"
OUT_DIR2 = "data/data2_"

In [4]:
df_train = load_csv(os.path.join(INP_DIR, "train_cleaned.csv"))
df_test = load_csv(os.path.join(INP_DIR, "test_cleaned.csv"))

/home/hai/opt/python_virtual_environments/xgboost/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Memory usage before changing types 6946.48 MB
Memory usage after changing types 3302.66 MB
Memory usage before changing types 116.20 MB
Memory usage after changing types 49.28 MB


In [5]:
# drop this column becuase it is too imbalanced
df_train = df_train.drop(["ind_empleado"], axis=1)
df_test = df_test.drop(["ind_empleado"], axis=1)

In [6]:
# days from when the data is recorded 
df_train["fecha_alta"] = (df_train["fecha_alta"] - df_train["fecha_dato"]).dt.days
df_test["fecha_alta"] = (df_test["fecha_alta"] - df_test["fecha_dato"]).dt.days

In [7]:
df_train.isnull().sum().sum(), df_test.isnull().sum().sum()

(0, 0)

In [8]:
df_train.shape, df_test.shape

((13647309, 67), (929615, 19))

In [9]:
df_train.columns

Index(['fecha_dato', 'ncodpers', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall',
       'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento',
       'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1',
       'ind_ahor_fin_ult1_NEW_PUR', 'ind_aval_fin_ult1_NEW_PUR',
       'ind_cco_fin_ult1_NEW_PUR', 'ind_cder_fin_ult1_NEW_PUR',
       'ind_cno_fin_ult1_NEW_PUR', 'ind_ctj

In [10]:
df_test.columns

Index(['fecha_dato', 'ncodpers', 'pais_residencia', 'sexo', 'age',
       'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall',
       'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento'],
      dtype='object')

In [11]:
PROD_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1$", col)]
print(PROD_COLS)

['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']


In [12]:
NEW_PURCH_COLS = [col for col in df_train.columns if re.match(r"^ind_.*_ult1_NEW_PUR$", col)]
print(NEW_PURCH_COLS, "\n")

prod_popul = df_train[NEW_PURCH_COLS].sum(axis=0)
prod_popul = prod_popul.sort_values(ascending=False)/prod_popul.sum() * 100
print(prod_popul, "\n")

NEW_PURCH_COLS = prod_popul.sort_values(ascending=False).index.to_list()
print(NEW_PURCH_COLS)

['ind_ahor_fin_ult1_NEW_PUR', 'ind_aval_fin_ult1_NEW_PUR', 'ind_cco_fin_ult1_NEW_PUR', 'ind_cder_fin_ult1_NEW_PUR', 'ind_cno_fin_ult1_NEW_PUR', 'ind_ctju_fin_ult1_NEW_PUR', 'ind_ctma_fin_ult1_NEW_PUR', 'ind_ctop_fin_ult1_NEW_PUR', 'ind_ctpp_fin_ult1_NEW_PUR', 'ind_deco_fin_ult1_NEW_PUR', 'ind_deme_fin_ult1_NEW_PUR', 'ind_dela_fin_ult1_NEW_PUR', 'ind_ecue_fin_ult1_NEW_PUR', 'ind_fond_fin_ult1_NEW_PUR', 'ind_hip_fin_ult1_NEW_PUR', 'ind_plan_fin_ult1_NEW_PUR', 'ind_pres_fin_ult1_NEW_PUR', 'ind_reca_fin_ult1_NEW_PUR', 'ind_tjcr_fin_ult1_NEW_PUR', 'ind_valo_fin_ult1_NEW_PUR', 'ind_viv_fin_ult1_NEW_PUR', 'ind_nomina_ult1_NEW_PUR', 'ind_nom_pens_ult1_NEW_PUR', 'ind_recibo_ult1_NEW_PUR'] 

ind_recibo_ult1_NEW_PUR      27.197280
ind_nom_pens_ult1_NEW_PUR    15.048197
ind_nomina_ult1_NEW_PUR      13.101134
ind_cco_fin_ult1_NEW_PUR     12.426018
ind_tjcr_fin_ult1_NEW_PUR    12.304237
ind_cno_fin_ult1_NEW_PUR      6.601516
ind_ecue_fin_ult1_NEW_PUR     4.682679
ind_dela_fin_ult1_NEW_PUR     2.2557

In [13]:
# total number of prod
df_train["TOTAL_PRODS"] = df_train[NEW_PURCH_COLS].sum(axis=1)

In [14]:
DEMOG_COLS = [col for col in df_train.columns 
              if col not in PROD_COLS + NEW_PURCH_COLS + ["fecha_dato", "ncodpers", "TOTAL_PRODS"]]
print(DEMOG_COLS)

['pais_residencia', 'sexo', 'age', 'fecha_alta', 'ind_nuevo', 'antiguedad', 'indrel', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada', 'indfall', 'cod_prov', 'ind_actividad_cliente', 'renta', 'segmento']


In [15]:
LAG_COLS = ["fecha_alta", "ind_nuevo", "antiguedad", "indrel", "tiprel_1mes", 
            "ind_actividad_cliente", "renta", "segmento", "TOTAL_PRODS"]

In [16]:
def _is_unique(x):
    return len(x) == x.nunique()


def _onehot_reduce(df):
    df = df.copy()
    values = df.values
    
    new_values = values.copy()
    for i in range(values.shape[0]):
        row = values[i]
        if row.sum() > 1:
            new_row = np.zeros([row.shape[0]], dtype=np.int)
            
            where_1 = np.where(row == 1)[0]
            np.random.shuffle(where_1)
            idx = where_1[0]
            new_row[idx] = 1
            
            new_values[i] = new_row
    
    new_df = pd.DataFrame(data=new_values, index=df.index, columns=df.columns)
    return new_df


def extract_subset(df, row_filter, cols):
    df = df.copy()
    return df.loc[row_filter, cols]


def extract_y_NOTUSED(df, timestamp, y_cols=NEW_PURCH_COLS):
    # only use row when customer buys exactly one product
    any_new_pur = df[y_cols].sum(axis=1) == 1
    row_filter = (df["fecha_dato"] == timestamp) & any_new_pur
    cols = ["ncodpers"] + y_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"
    
    df_out.columns = [col.replace("_NEW_PUR", "") if col.endswith("_NEW_PUR") else col for col in df_out.columns]
    
    return df_out


def extract_y(df, timestamp, y_cols=NEW_PURCH_COLS):
    # only use row when customer buys at least one product
    any_new_pur = df[y_cols].sum(axis=1) > 0
    row_filter = (df["fecha_dato"] == timestamp) & any_new_pur
    cols = ["ncodpers"] + y_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"
    
    df_out.columns = [col.replace("_NEW_PUR", "") if col.endswith("_NEW_PUR") else col for col in df_out.columns]
    
    id_col = df_out["ncodpers"]
    pur_cols = [col for col in df_out.columns if col != "ncodpers"]
    df_out = _onehot_reduce(df_out[pur_cols])
    df_out["ncodpers"] = id_col
    
    return df_out


def extract_x_demog(df, timestamp, customer_ids, demog_cols=DEMOG_COLS):
    row_filter = (df["fecha_dato"] == timestamp) & df["ncodpers"].isin(customer_ids)
    cols = ["ncodpers"] + demog_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"

    return df_out


# This function can give dataframe having less rows than len(customer_ids)
# This is becuse number of row at lag_timestamp is not the same as at timestamp
def extract_lag_features(df, timestamp_lag, customer_ids, suffix="_LAG", 
                         sel_cols=PROD_COLS+LAG_COLS):
    row_filter = (df["fecha_dato"] == timestamp_lag) & df["ncodpers"].isin(customer_ids)
    cols = ["ncodpers"] + sel_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"
    
    df_out.columns = ["ncodpers"] + [col + suffix for col in df_out.columns if col != "ncodpers"]
    
    return df_out



def extract_X_y_train(df, timestamp, timestamp_lags):
    y_train = extract_y(df, timestamp)
    print("y_train.shape", y_train.shape)
    
    x_demog = extract_x_demog(df, timestamp, customer_ids=y_train["ncodpers"])
    print("x_demog.shape:", x_demog.shape)
    assert y_train.shape[0] == x_demog.shape[0], "x_demog must have the same number of rows as y_train"
    
    x_lags = []
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        x_lag = extract_lag_features(df, lag, customer_ids=y_train["ncodpers"], suffix=lag_label)
        print(lag, lag_label, x_lag.shape)
        x_lags.append(x_lag)
    
    X_train = y_train[["ncodpers"]].merge(x_demog, how="left", on="ncodpers")
    print("Nulls after merging y and x_demog:", X_train.isnull().sum().sum())
    
    for t, x_lag in enumerate(x_lags):
        X_train = X_train.merge(x_lag, how="left", on="ncodpers")
        print("Nulls at %d:" %(t + 1), X_train.isnull().sum().sum())
    
    print("X_train.shape:", X_train.shape)
    print("y_train.shape:", y_train.shape)
    return X_train, y_train


def extract_X_test(train, test, timestamp, timestamp_lags):
    x_demog = extract_x_demog(test, timestamp, customer_ids=test["ncodpers"])
    print("x_demog.shape:", x_demog.shape)
    print("Nulls of x_demog:", x_demog.isnull().sum().sum())
    
    x_lags = []
    for t, lag in enumerate(timestamp_lags):
        assert pd.to_datetime(lag) < pd.to_datetime(timestamp), lag + " lag is not before timestamp " +  timestamp
        lag_label = "_LAG%d" % (t + 1)
        
        x_lag = extract_lag_features(train, lag, customer_ids=test["ncodpers"], suffix=lag_label)
        print(lag, lag_label, x_lag.shape)
        x_lags.append(x_lag)
        
    X_test = x_demog
    for t, x_lag in enumerate(x_lags):
        X_test = X_test.merge(x_lag, how="left", on="ncodpers")
        print("Nulls at %d:" %(t + 1), X_test.isnull().sum().sum())
    
    print("X_test.shape:", X_test.shape)
    return X_test

In [17]:
def extract_x_demog(df, timestamp, customer_ids, demog_cols=DEMOG_COLS):
    row_filter = (df["fecha_dato"] == timestamp) & df["ncodpers"].isin(customer_ids)
    cols = ["ncodpers"] + demog_cols
    
    df_out = extract_subset(df, row_filter, cols)
    assert _is_unique(df_out["ncodpers"]), "ncodpers must be unique"

    return df_out

In [18]:
df_train["fecha_dato"].unique()

array(['2015-01-28T00:00:00.000000000', '2015-02-28T00:00:00.000000000',
       '2015-03-28T00:00:00.000000000', '2015-04-28T00:00:00.000000000',
       '2015-05-28T00:00:00.000000000', '2015-06-28T00:00:00.000000000',
       '2015-07-28T00:00:00.000000000', '2015-08-28T00:00:00.000000000',
       '2015-09-28T00:00:00.000000000', '2015-10-28T00:00:00.000000000',
       '2015-11-28T00:00:00.000000000', '2015-12-28T00:00:00.000000000',
       '2016-01-28T00:00:00.000000000', '2016-02-28T00:00:00.000000000',
       '2016-03-28T00:00:00.000000000', '2016-04-28T00:00:00.000000000',
       '2016-05-28T00:00:00.000000000'], dtype='datetime64[ns]')

# Extract `X_2016_04`, `y_2016_04`

In [19]:
timestamp = "2016-04-28"
timestamp_lags = ["2016-03-28", "2016-02-28", "2016-01-28", 
                  "2015-12-28", "2015-11-28", "2015-10-28", 
                  "2015-09-28", "2015-08-28", "2015-07-28", 
                  "2015-06-28", "2015-05-28", "2015-04-28"]

X_2016_04, y_2016_04 = extract_X_y_train(df_train, timestamp, timestamp_lags)

y_train.shape (26791, 25)
x_demog.shape: (26791, 18)
2016-03-28 _LAG1 (26736, 34)
2016-02-28 _LAG2 (25051, 34)
2016-01-28 _LAG3 (24255, 34)
2015-12-28 _LAG4 (23556, 34)
2015-11-28 _LAG5 (22805, 34)
2015-10-28 _LAG6 (22308, 34)
2015-09-28 _LAG7 (21819, 34)
2015-08-28 _LAG8 (21415, 34)
2015-07-28 _LAG9 (21143, 34)
2015-06-28 _LAG10 (20223, 34)
2015-05-28 _LAG11 (20064, 34)
2015-04-28 _LAG12 (19885, 34)
Nulls after merging y and x_demog: 0
Nulls at 1: 1815
Nulls at 2: 59235
Nulls at 3: 142923
Nulls at 4: 249678
Nulls at 5: 381216
Nulls at 6: 529155
Nulls at 7: 693231
Nulls at 8: 870639
Nulls at 9: 1057023
Nulls at 10: 1273767
Nulls at 11: 1495758
Nulls at 12: 1723656
X_train.shape: (26791, 414)
y_train.shape: (26791, 25)


In [20]:
X_2016_04.head()

,ncodpers,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,...,ind_recibo_ult1_LAG12,fecha_alta_LAG12,ind_nuevo_LAG12,antiguedad_LAG12,indrel_LAG12,tiprel_1mes_LAG12,ind_actividad_cliente_LAG12,renta_LAG12,segmento_LAG12,TOTAL_PRODS_LAG12
0,15925,ES,V,63.0,-7773,False,255.0,A,1,A,...,0.0,-7407.0,False,246.0,A,A,True,42831.691406,02 - PARTICULARES,1.0
1,15926,ES,V,56.0,-7773,False,253.0,A,1,A,...,1.0,-7407.0,False,244.0,A,A,True,128376.242188,02 - PARTICULARES,0.0
2,15950,ES,V,61.0,-7773,False,255.0,A,1,A,...,1.0,-7407.0,False,246.0,A,A,True,141979.265625,02 - PARTICULARES,0.0
3,15965,ES,H,52.0,-7773,False,255.0,A,1,A,...,1.0,-7407.0,False,246.0,A,A,True,44353.171875,01 - TOP,1.0
4,15969,ES,H,56.0,-7773,False,255.0,A,1,A,...,1.0,-7407.0,False,246.0,A,A,True,771167.187500,02 - PARTICULARES,0.0


In [21]:
y_2016_04.head()

,ind_recibo_ult1,ind_nom_pens_ult1,ind_nomina_ult1,ind_cco_fin_ult1,ind_tjcr_fin_ult1,ind_cno_fin_ult1,ind_ecue_fin_ult1,ind_dela_fin_ult1,ind_reca_fin_ult1,ind_ctma_fin_ult1,...,ind_plan_fin_ult1,ind_ctju_fin_ult1,ind_deme_fin_ult1,ind_pres_fin_ult1,ind_cder_fin_ult1,ind_hip_fin_ult1,ind_viv_fin_ult1,ind_aval_fin_ult1,ind_ahor_fin_ult1,ncodpers
521,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15925
538,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15926
896,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15950
1100,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15965
1134,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15969


In [22]:
X_2016_04.to_csv(os.path.join(OUT_DIR1, "X_2016_04.csv"), index=False)
y_2016_04.to_csv(os.path.join(OUT_DIR1, "y_2016_04.csv"), index=False)

# Extract `X_2016_05`, `y_2016_05`

In [23]:
timestamp = "2016-05-28"
timestamp_lags = ["2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28",
                  "2015-12-28", "2015-11-28", "2015-10-28", 
                  "2015-09-28", "2015-08-28", "2015-07-28", 
                  "2015-06-28", "2015-05-28"]

X_2016_05, y_2016_05 = extract_X_y_train(df_train, timestamp, timestamp_lags)

y_train.shape (27916, 25)
x_demog.shape: (27916, 18)
2016-04-28 _LAG1 (27875, 34)
2016-03-28 _LAG2 (26372, 34)
2016-02-28 _LAG3 (25692, 34)
2016-01-28 _LAG4 (24936, 34)
2015-12-28 _LAG5 (23968, 34)
2015-11-28 _LAG6 (23553, 34)
2015-10-28 _LAG7 (23179, 34)
2015-09-28 _LAG8 (22725, 34)
2015-08-28 _LAG9 (22315, 34)
2015-07-28 _LAG10 (22053, 34)
2015-06-28 _LAG11 (21110, 34)
2015-05-28 _LAG12 (20963, 34)
Nulls after merging y and x_demog: 0
Nulls at 1: 1353
Nulls at 2: 52305
Nulls at 3: 125697
Nulls at 4: 224037
Nulls at 5: 354321
Nulls at 6: 498300
Nulls at 7: 654621
Nulls at 8: 825924
Nulls at 9: 1010757
Nulls at 10: 1204236
Nulls at 11: 1428834
Nulls at 12: 1658283
X_train.shape: (27916, 414)
y_train.shape: (27916, 25)


In [24]:
X_2016_05.head()

,ncodpers,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,...,ind_recibo_ult1_LAG12,fecha_alta_LAG12,ind_nuevo_LAG12,antiguedad_LAG12,indrel_LAG12,tiprel_1mes_LAG12,ind_actividad_cliente_LAG12,renta_LAG12,segmento_LAG12,TOTAL_PRODS_LAG12
0,15889,ES,V,56.0,-7803,False,255.0,A,1,A,...,0.0,-7437.0,False,245.0,A,A,True,326124.90625,01 - TOP,1.0
1,15929,ES,H,71.0,-7803,False,256.0,A,1,A,...,1.0,-7437.0,False,246.0,A,A,True,289565.06250,02 - PARTICULARES,1.0
2,15952,ES,V,46.0,-7803,False,256.0,A,1,A,...,1.0,-7437.0,False,246.0,A,A,True,297068.12500,02 - PARTICULARES,0.0
3,15988,ES,V,63.0,-7803,False,256.0,A,1,A,...,0.0,-7437.0,False,246.0,A,A,True,353419.37500,02 - PARTICULARES,0.0
4,15993,ES,V,55.0,-7803,False,250.0,A,1,A,...,1.0,-7437.0,False,240.0,A,A,True,139070.96875,01 - TOP,0.0


In [25]:
y_2016_05.head()

,ind_recibo_ult1,ind_nom_pens_ult1,ind_nomina_ult1,ind_cco_fin_ult1,ind_tjcr_fin_ult1,ind_cno_fin_ult1,ind_ecue_fin_ult1,ind_dela_fin_ult1,ind_reca_fin_ult1,ind_ctma_fin_ult1,...,ind_plan_fin_ult1,ind_ctju_fin_ult1,ind_deme_fin_ult1,ind_pres_fin_ult1,ind_cder_fin_ult1,ind_hip_fin_ult1,ind_viv_fin_ult1,ind_aval_fin_ult1,ind_ahor_fin_ult1,ncodpers
16,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15889
590,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15929
931,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15952
1412,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15988
1479,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15993


In [26]:
X_2016_05.to_csv(os.path.join(OUT_DIR1, "X_2016_05.csv"), index=False)
y_2016_05.to_csv(os.path.join(OUT_DIR1, "y_2016_05.csv"), index=False)

# Extract `X_2016_06`

In [27]:
timestamp = "2016-06-28"
timestamp_lags = ["2016-05-28", "2016-04-28", "2016-03-28", "2016-02-28", "2016-01-28",
                  "2015-12-28", "2015-11-28", "2015-10-28", "2015-09-28", 
                  "2015-08-28", "2015-07-28", "2015-06-28"]

X_2016_06 = extract_X_test(df_train, df_test, timestamp, timestamp_lags)

x_demog.shape: (929615, 18)
Nulls of x_demog: 0
2016-05-28 _LAG1 (929615, 34)
2016-04-28 _LAG2 (925252, 34)
2016-03-28 _LAG3 (920975, 34)
2016-02-28 _LAG4 (915679, 34)
2016-01-28 _LAG5 (909885, 34)
2015-12-28 _LAG6 (903429, 34)
2015-11-28 _LAG7 (896458, 34)
2015-10-28 _LAG8 (881573, 34)
2015-09-28 _LAG9 (854574, 34)
2015-08-28 _LAG10 (832230, 34)
2015-07-28 _LAG11 (818424, 34)
2015-06-28 _LAG12 (622404, 34)
Nulls at 1: 0
Nulls at 2: 143979
Nulls at 3: 429099
Nulls at 4: 888987
Nulls at 5: 1540077
Nulls at 6: 2404215
Nulls at 7: 3498396
Nulls at 8: 5083782
Nulls at 9: 7560135
Nulls at 10: 10773840
Nulls at 11: 14443143
Nulls at 12: 24581106
X_test.shape: (929615, 414)


In [28]:
X_2016_06.head()

,ncodpers,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,indrel_1mes,tiprel_1mes,...,ind_recibo_ult1_LAG12,fecha_alta_LAG12,ind_nuevo_LAG12,antiguedad_LAG12,indrel_LAG12,tiprel_1mes_LAG12,ind_actividad_cliente_LAG12,renta_LAG12,segmento_LAG12,TOTAL_PRODS_LAG12
0,15889,ES,V,56.0,-7834,False,256.0,A,1,A,...,0.0,-7468.0,False,245.0,A,A,True,326124.906250,01 - TOP,0.0
1,1170544,ES,H,36.0,-1035,False,34.0,A,1,I,...,0.0,-669.0,False,23.0,A,A,False,67526.281250,02 - PARTICULARES,0.0
2,1170545,ES,V,22.0,-1035,False,34.0,A,1,A,...,0.0,-669.0,False,23.0,A,A,False,97689.296875,03 - UNIVERSITARIO,0.0
3,1170547,ES,H,22.0,-1035,False,34.0,A,1,I,...,0.0,-669.0,False,23.0,A,I,False,148402.984375,03 - UNIVERSITARIO,0.0
4,1170548,ES,H,22.0,-1035,False,34.0,A,1,I,...,0.0,-669.0,False,23.0,A,I,False,106885.796875,03 - UNIVERSITARIO,0.0


In [29]:
X_2016_06.to_csv(os.path.join(OUT_DIR1, "X_2016_06.csv"), index=False)